In [1]:
import torch
import numpy as np
import pandas as pd
from transformers import LlamaTokenizer, LlamaForSequenceClassification, LlamaModel, LlamaForCausalLM, AutoConfig, AutoModel
from accelerate import infer_auto_device_map, init_empty_weights

from src.utils import *
from datasets import Dataset
from src.models.baseline import Baseline
from torch.nn.functional import one_hot


c:\Users\marco\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
RANDOM_SEED=42

url = "https://drive.google.com/uc?export=download&id=1wVNU2XvvhqjaGXZM-JLJwOt97gt4g9j2"
dataset_name = "MELD_train_efr.json"

df_manager = DataframeManager(url, dataset_name)

df = df_manager.produce_df()
df

Current work directory: d:\shared\unibo\year2\NLP\assignments\project


,episode,emotions,utterances,triggers,emotions_id
0,utterance_0,"[neutral, neutral, neutral, neutral, surprise]",[also I was the point person on my company's t...,"[0, 0, 0, 1, 0]","[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 0...."
1,utterance_1,"[neutral, neutral, neutral, neutral, surprise,...",[also I was the point person on my company's t...,"[0, 0, 0, 0, 0, 1, 0]","[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 0...."
2,utterance_2,"[neutral, neutral, neutral, neutral, surprise,...",[also I was the point person on my company's t...,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0]","[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 0...."
3,utterance_3,"[neutral, neutral, neutral, neutral, surprise,...",[also I was the point person on my company's t...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]","[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 0...."
4,utterance_4,"[surprise, sadness, surprise, fear]",[But then who? The waitress I went out with la...,"[0, 0, 1, 0]","[[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0], [0.0, 0...."
...,...,...,...,...,...
3995,utterance_3995,"[neutral, joy, neutral, neutral, surprise, dis...","[Hey., Hey!, So how was Joan?, I broke up with...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]","[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 1...."
3996,utterance_3996,"[neutral, joy, neutral, neutral, surprise, dis...","[Hey., Hey!, So how was Joan?, I broke up with...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]","[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 1...."
3997,utterance_3997,"[neutral, joy, neutral, neutral, surprise, dis...","[Hey., Hey!, So how was Joan?, I broke up with...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]","[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 1...."
3998,utterance_3998,"[neutral, joy, neutral, neutral, surprise, dis...","[Hey., Hey!, So how was Joan?, I broke up with...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]","[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 1...."


In [4]:
train_df, val_df, test_df = df_manager.split_df(RANDOM_SEED)
print(train_df.shape)
print(val_df.shape)
print(test_df.shape)

(3200, 5)
(400, 5)
(400, 5)


In [10]:
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

In [12]:
train_dataset[1]

{'episode': 'utterance_3491',
 'emotions': ['surprise', 'fear', 'surprise', 'sadness'],
 'utterances': ['You-you\x85you had sex with Ursula?!',
  'Uh, a little bit. She-she-she walked in and I thought she was you and I kissed her and',
  "You didn't notice she was wearing different clothes?!",
  'Well I was just so excited to see you.'],
 'triggers': [0, 0, 1, 0],
 'emotions_id': [6, 5, 6, 0],
 '__index_level_0__': 3491}

In [2]:
model_path = 'openlm-research/open_llama_3b_v2'
tokenizer = LlamaTokenizer.from_pretrained(model_path)

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [7]:
# config = AutoConfig.from_pretrained(model_path)
# with init_empty_weights():
#     model = AutoModel.from_config(config)
# device_map = infer_auto_device_map(model)

model = LlamaModel.from_pretrained(
    model_path, torch_dtype=torch.float16, device_map='auto', offload_folder="offload_base", offload_state_dict=True
)

In [9]:
prompt = 'Q: What is the largest animal?\nA:'
input_ids = tokenizer(prompt, return_tensors="pt").input_ids

output = model.generate(input_ids)
# output = class_model(input_ids)

# generation_output = causal_model.generate(
#     input_ids=input_ids, max_new_tokens=32
# )
# print(tokenizer.decode(generation_output[0]))

TypeError: The current model class (LlamaModel) is not compatible with `.generate()`, as it doesn't have a language model head. Please use one of the following classes instead: {'LlamaForCausalLM'}

https://huggingface.co/blog/accelerate-large-models